### Launch the ROS demo

In [ ]:
%%bash --bg
trap 'kill $(jobs -p)' EXIT
xterm -e /bin/bash -l -c "roslaunch demo.launch"

Create the controller and run it periodically in a thread

In [ ]:
import rospy
from threading import Thread
from controller import Controller

rospy.init_node('ns_demo')
c = Controller()

def worker():
    rate = rospy.Rate(50)
    while not rospy.is_shutdown():
        c.pose_control(c.im_server.target)
        rate.sleep()
    
t = Thread(target=worker)
t.start()

Create a list of slider widgets, one for each joint, to chose the default pose

In [ ]:
import ipywidgets
from ipywidgets import FloatSlider, Layout, Button, Box
joint_widgets = [FloatSlider(min = j.min, max = j.max, step = (j.max-j.min) / 100, description = j.name) \
                 for j in c.robot.active_joints]
joint_weights = [FloatSlider(min = 0, max = 1, step = 0.01, value = 1, description = j.name) \
                 for j in c.robot.active_joints]
task_dimensions = ['x', 'y', 'z', 'rx', 'ry', 'rz']
task_weights = [FloatSlider(min = 0, max = 1, step = 0.01, value = 1, description = name) \
                for name in task_dimensions]

React to slider (value) changes by adapting the default joint pose

In [ ]:
def on_joint_preference(event):
    widget = event.owner
    c.preferred_joints[c.joint_msg.name.index(widget.description)] = widget.value

for widget in joint_widgets:
    widget.observe(on_joint_preference, 'value')

In [ ]:
def on_joint_weight(event):
    widget = event.owner
    c.joint_weights[c.joint_msg.name.index(widget.description)] = widget.value

for widget in joint_weights:
    widget.observe(on_joint_weight, 'value')

In [ ]:
def on_task_weight(event):
    widget = event.owner
    c.cartesian_weights[task_dimensions.index(widget.description)] = widget.value

for widget in task_weights:
    widget.observe(on_task_weight, 'value')

Collect all widgets (sliders and buttons) in a form and display them

In [ ]:
layout = Layout(display='flex', flex_flow='column', border='solid 2px', align_items='stretch')
form = Box([Box(w, layout=layout) for w in [joint_widgets, joint_weights, task_weights]])
display(form)